In [0]:
# Instalacja pakietow
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.6MB/s 


In [0]:
# Importy
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
# Zaczytanie danych
df=pd.read_hdf('data/car.h5')
df.shape
# Usuwanie cen w euro. Lepszym rozwiazaniem bylaby normalizacja, jednak samochody o cenach w euro stanowia mniej niz 0,2% przypadkow(patrz: day3)
df=df[df['price_currency']!='EUR']


## Feature engineering

In [0]:
# Indeksowanie wartosci
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_values
  else:
    df[feat+SUFFIX_CAT]=factorized_values

In [0]:
# Selekcja cech:
# Tylko cechy ktorym zostaly przypisane indeksy
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
# Prosty model
X = df[cat_feats].values
y = df['price_value'].values
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19566.588937368324, 90.61814865165921)

In [0]:
# Funkcja zwracajaca wyniki w zaleznosci od modelu i zestawu danych
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  # model = DecisionTreeRegressor(max_depth=5)
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

## Random Forest

In [0]:
model=RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18734.2072708522, 109.87074106274046)

## XGBoost

In [0]:
xgb_params ={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
    }

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[20:29:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:29:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:29:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13039.290196724838, 109.36715375706265)

In [0]:
# Istotnosc cech wg xgboost
m=xgb.XGBRegressor(max_depth=5, n_etimators=50, learining_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[20:33:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1434 ± 0.0028,param_rok-produkcji__cat
0.1056 ± 0.0011,param_stan__cat
0.0964 ± 0.0021,param_napęd__cat
0.0505 ± 0.0017,param_skrzynia-biegów__cat
0.0450 ± 0.0014,param_marka-pojazdu__cat
0.0420 ± 0.0012,param_moc__cat
0.0404 ± 0.0012,param_faktura-vat__cat
0.0214 ± 0.0004,param_typ__cat
0.0172 ± 0.0006,param_pojemność-skokowa__cat
0.0167 ± 0.0010,param_kod-silnika__cat


In [0]:
feats=['param_rok-produkcji__cat','param_stan__cat','param_napęd__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_moc__cat',
'param_faktura-vat__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

In [0]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[20:50:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:50:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13296.763485002144, 90.68056245368918)

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats=['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_moc__cat',
'param_faktura-vat__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[20:55:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:56:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11019.297038651634, 113.86932883713467)

In [50]:
# df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
feats=['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_moc',
'param_faktura-vat__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[21:32:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:32:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:32:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9311.478476635679, 75.86114898927406)

In [52]:
 #df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))
 df['param_pojemność-skokowa'].unique
feats=['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_moc',
'param_faktura-vat__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'param_kod-silnika__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

model = xgb.XGBRegressor(**xgb_params)
# run_model(model, feats)

[21:34:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9178.232622088182, 114.87103984174288)